# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/'

fileName = "rice.png"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

img = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)
plt.imshow(img, 'gray')

In [ ]:

def adaptive_bin(img, W):
    helper = img.copy()
    helper = helper*0

    for i in range(0,len(img),W):
        for j in range(0,len(img[i]),W):
            helper[i][j] = np.mean([row[j:(j+W)] for row in img[i:(i+W)]])

    window_cnt = -1

    binarized = img.copy()

    for i in range(0,len(img)):
        for j in range(0,len(img[i])):
            binarized[i][j] = binarized[i][j] > helper[(i//W)*W][(j//W)*W]
    
    return binarized

plt.axis('off')
plt.imshow(adaptive_bin(img, 16), 'gray')

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego piksela wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
def interp_adaptive_bin(img, W):
    helper = img.copy()
    helper = helper*0
    for i in range(0,len(img),W):
        for j in range(0,len(img[i]),W):
            helper[i//W][j//W] = np.mean([row[j:(j+W)] for row in img[i:(i+W)]])

    binarized = img.copy()

    for i in range(0,len(img)):
        for j in range(0,len(img[i])):
    #         binarized[i][j] = binarized[i][j] > helper[window_cnt*W][window_cnt*W]


            # RED SQUARE
            if(i <= W/2 or i >= len(img)-W/2):
                if(j <= W/2 or j >= len(img)-W/2):
                    binarized[i][j] = img[i][j] > helper[(i//W)][(j//W)]

            # GREEN SQUARE
            # VERTICAL
            if(i >= W/2 and i <= len(img)-W/2):
                if(j < W/2 or j > len(img)-W/2):
                    if(((i//W)+1)*W - W/2 < i):
                        binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)] + helper[((i//W)+1)][(j//W)])/2
                    else:
                        binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)] + helper[((i//W)-1)][(j//W)])/2
            # HORIZONTAL
            if(j >= W/2 and j <= len(img)-W/2):
                if(i <= W/2 or i >= len(img)-W/2):
                    if(((j//W)+1)*W - W/2 < j):
                        binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)] + helper[((i//W))][(j//W)+1])/2
                    else:
                        binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)] + helper[((i//W))][(j//W)-1])/2       
            # BLUE SQUARE
            if(i >= W/2 and i <= len(img)-W/2):
                if(j >= W/2 and j <= len(img)-W/2):
                    # LOWER HALF
                    if(((i//W)+1)*W - W/2 <= i):
                        # RIGHT HALF
                        if(((j//W)+1)*W - W/2 <= j):
                            binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)]*2 + helper[((i//W))][(j//W)+1] + helper[((i//W)+1)][(j//W)])/4
                        # LEFT HALF
                        else:
                            binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)]*2 + helper[((i//W))][(j//W)-1] + helper[((i//W)+1)][(j//W)])/4
                    # UPPER HALF
                    else:
                        # RIGHT HALF
                        if(((j//W)+1)*W - W/2 <= j):
                            binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)]*2 + helper[((i//W))][(j//W)+1] + helper[((i//W)-1)][(j//W)])/4
                        # LEFT HALF:
                        else:
                            binarized[i][j] = img[i][j] > (helper[(i//W)][(j//W)]*2 + helper[((i//W))][(j//W)-1] + helper[((i//W)-1)][(j//W)])/4


    return binarized
   

plt.axis('off')
plt.imshow(interp_adaptive_bin(img, 16), 'gray', vmin=0, vmax=1)

In [ ]:
def otsu(img):
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    hist = np.squeeze(hist)
    chist = np.cumsum(hist)
    hist = hist/chist[len(chist)-1]
    chist = chist/chist[len(chist)-1]

    k = int(np.sum(np.dot(hist, range(0,256))))
    
    b = np.zeros(256)
    m_0 = 0
    
    k_max = -1
    value_max = -1
    for i in range(0,255):
        if(chist[i] > 0 and chist[i] < 1):
            m_0 = m_0 + i*hist[i]
            b[i] = ((k*chist[i] - m_0)**2)/(chist[i]*(1-chist[i]))
            if b[i] > value_max:
                k_max = i
                value_max = b[i]
            
    return img > k_max

In [ ]:
def bin_local_mean(img, window_size):
    (X,Y) = img.shape
    W = window_size
    new_img = img.copy()
    
    for j in range(int(W/2), int(Y-W/2)):
        for i in range(int(W/2), int(X-W/2)):
            window = [row[i-int(W/2):i+int(W/2)] for row in img[j-int(W/2):j+int(W/2)]]
            new_img[i,j] = img[i, j] > np.mean(window)
            
    return new_img

In [ ]:
def bin_local_sauvoli(img, window_size, R, k):
    (X,Y) = img.shape
    W = window_size
    new_img = img.copy()
    
    for j in range(int(W/2), int(Y-W/2)):
        for i in range(int(W/2), int(X-W/2)):
            window = [row[i-int(W/2):i+int(W/2)] for row in img[j-int(W/2):j+int(W/2)]]
            improvement = k*(np.std(window)/R -1)
            if(improvement < 0):
                new_img[i,j] = img[i, j] > np.mean(window)*(1-improvement)
            else:
                new_img[i,j] = img[i, j] > np.mean(window)*(1+improvement)
            
    return new_img

In [ ]:
fig, axs = plt.subplots(1,6)
fig.set_size_inches(20, 10)
axs[0].axis('off')
axs[0].imshow(img, 'gray')
axs[0].set_title('Oryginalny obraz')
axs[1].axis('off')
axs[1].imshow(otsu(img),  'gray', vmin=0, vmax=1)
axs[1].set_title('Otsu')
axs[2].axis('off')
axs[2].imshow(bin_local_mean(img, 15),  'gray', vmin=0, vmax=1)
axs[2].set_title('Lokalna na średniej')
axs[3].axis('off')
axs[3].imshow(bin_local_sauvoli(img, 15, 128, 0.15),  'gray', vmin=0, vmax=1)
axs[3].set_title('Lokalna Sauvoli')
axs[4].axis('off')
axs[4].imshow(adaptive_bin(img, 16),  'gray', vmin=0, vmax=1)
axs[4].set_title('Oknem')
axs[5].axis('off')
axs[5].imshow(interp_adaptive_bin(img, 16),  'gray', vmin=0, vmax=1)
axs[5].set_title('Oknem z interpolacją')

Najlepszy wynik dała binaryzacja Otsu. </br>
Dla binaryzacji lokalnej, w obu przypadkach pojawił się problem z oświetleniem oraz drobne artefakty. </br>
Dla binaryzacji z oknem zniknął problem z oświetleniem, jednak dla okna bez interpolacji zwiększyła się liczba artefaktów, która została zmniejszona w binaryzacji z interpolacją.